In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 2023_parenting_concern_posts.csv to 2023_parenting_concern_posts.csv


In [ ]:
import pandas as pd
import io
df_2023 = pd.read_csv(io.BytesIO(uploaded["2023_parenting_concern_posts.csv"]))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
from google.colab import files
uploaded1 = files.upload()

Saving all_parenting_concern_posts.csv to all_parenting_concern_posts (1).csv


In [ ]:
import pandas as pd
import io
df_all = pd.read_csv(io.BytesIO(uploaded1["all_parenting_concern_posts.csv"]))

In [ ]:
from google.colab import files
uploaded2 = files.upload()

Saving stopwords-ja.txt to stopwords-ja.txt


In [ ]:
io.StringIO(uploaded2["stopwords-ja.txt"].decode("utf-8"))

In [ ]:
print(df_all.head(10))

# BERTopic

In [ ]:
# ----------
# BERTopic
!pip install bertopic
from bertopic import BERTopic
#from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import locale
locale.getpreferredencoding = lambda: "UTF-8"

! pip install mecab-python3 unidic-lite
import MeCab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached bertopic-0.15.0-py2.py3-none-any.whl (143 kB)
  Using cached hdbscan-0.8.29.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached umap-learn-0.5.3.tar.gz (88 kB)
  Preparing metadata (setup.py) ... done
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached huggingface_hub-0.15.1-py3-none-any.whl (236 kB)
  Using cached pynndescent-0.5.10.tar.gz (1.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached safetensors-0.3.1-cp310-cp310-manyli

In [ ]:
#docs = df_all['text_clean'].tolist()
docs = df_2023['text_clean'].tolist()
#docs = df_2023['summary_clean'].tolist()
print(len(docs))


def tokenize_jp(docs):
    # Read stopwords from file
    with open('stopwords-ja.txt', encoding='utf-8') as f:
        stopwords = set(f.read().split())
    #for doc in docs:
    words = MeCab.Tagger("-Owakati").parse(docs).split()
    # Remove stopwords from words
    words = [w for w in words if w not in stopwords]
    return words

vectorizer = CountVectorizer(tokenizer=tokenize_jp)

# Pass the above models to be used in BERTopic
model = BERTopic(
    embedding_model="paraphrase-multilingual-MiniLM-L12-v2", #ここ変更
    #embedding_model="all-MiniLM-L6-v2",
    vectorizer_model = vectorizer,
    language="japanese",
    calculate_probabilities=True,
    verbose=True,
    #nr_topics="auto"
    nr_topics="20"
    #umap_model=umap_model,
    #hdbscan_model=hdbscan_model,
    )



2765


In [ ]:
topics, probs = model.fit_transform(docs)

Batches:   0%|          | 0/87 [00:00<?, ?it/s]

2023-06-19 04:28:41,225 - BERTopic - Transformed documents to Embeddings
2023-06-19 04:29:01,009 - BERTopic - Reduced dimensionality
2023-06-19 04:29:01,475 - BERTopic - Clustered reduced embeddings
2023-06-19 04:29:11,353 - BERTopic - Reduced number of topics from 42 to 31


In [ ]:
model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,1249,-1_子供_人_たら_自分,"[子供, 人, たら, 自分, 夫, 仕事, いい, 思い, 時, 歳]",[義母 75歳 勿体無い精神がすごく強くてちょっとどうすればいいか困っています 旦那さんは...
1,0,423,0_子供_人_自分_時,"[子供, 人, 自分, 時, 母, 事, 言っ, たら, 今, てる]",[無責任で自己中すぎる親のせいで子供の頃は5人家族分の家事を全て1人でやって まともな人生を...
2,1,208,1_仕事_万_転職_社員,"[仕事, 万, 転職, 社員, 人, 会社, 年, 1, 勤務, 歳]",[働き方について悩んでいます 新婚27歳です 夫の転職をきっかけに前の会社を退職し 現在は...
3,2,132,2_月_休業_給付_金,"[月, 休業, 給付, 金, 育休, 日, 育児, 手当, 保険, 出産]",[以下引用して載せています パートの育児休業給付金の受給資格 条件 この制度は扶養内で...
4,3,117,3_時間_仕事_日_時,"[時間, 仕事, 日, 時, 旦那, 休み, 育児, 子供, 家事, 夫]",[夫婦仲についてです 長文ですみません 私 専業主婦 旦那 日曜のみ休み時々2日連休あり...
5,4,79,4_化_少子_対策_日本,"[化, 少子, 対策, 日本, 次元, 異, 支援, 子育て, 税, 政策]","[子育て支援異次元の少子化対策は課題が多いですか , 高校生と中学生の子供がいます 子供が小..."
6,5,61,5_母乳_ミルク_授乳_量,"[母乳, ミルク, 授乳, 量, 1, 日, 回, 体重, 完, 時間]",[母乳育児について相談です 先月出産して 1ヶ月半になります 母乳をなかなか吸えなくて助...
7,6,58,6_妻_主婦_夫_専業,"[妻, 主婦, 夫, 専業, 家事, 育児, てる, 自分, 嫁, 人]",[正社員で共働きしてくれる嫁のいる家庭が羨ましいです うちの嫁は育休後 私に相談もなく勝...
8,7,48,7_女性_男性_男_結婚,"[女性, 男性, 男, 結婚, 的, 仕事, 代, 自分, 家事, 年収]",[花嫁修行と家事手伝いと無職の違いは何 youtube動画で 結婚したい 専業主婦になりた...
9,8,40,8_土地_市_建て_家,"[土地, 市, 建て, 家, 住ん, 地域, 駅, 建てる, 地, 平屋]",[引っ越し先の物件で迷っています 現在 もうすぐ2歳の子と夫の3人で暮らしています 2...


In [ ]:
model.get_representative_docs(topic=-1)

['義母 75歳 勿体無い精神がすごく強くてちょっとどうすればいいか困っています  旦那さんは末っ子長男で姉が2人 子供も各1人ずつ男の子がいるのですが 旦那さん側の方はオモチャも子供服 下着も含め 全てサイズアウトしたものはお下がりを回しまくる感じが普通みたいです  私も服やオモチャを頂いたりしてありがたく使えるものは使わせて頂いてるのですが 流石にパンツは   という気持ちです  私共の子供は1歳4ヶ月の男の子で 現在第二子を妊娠中 旦那さんが子供を連れて2人で旦那の実家に泊まってくるのは有り難いのですが 旦那さんのお義母さんがオムツが勿体無いから 暑くなってきたのでこっちに来てる時はパンツ履かせると言っておられます  もちろんパンツはお古のもの  私からしたら何回も濡れた服やパンツを洗って手間がかかる方がよっぽど非効率すぎてっていう感じです  ましてや替えの服もその分たくさん持って行かないと行けないだろうし    そのパンツもお義姉さんの子供達の物だけではなく お義姉さんのご主人の甥っ子のものであったり 友達や知り合いの子供さんの物であったり   誰のか分からないので 洗ってあるとはいえやっぱり黄ばみとかあると凄く抵抗感があり    旦那さんともしっかり相談してパンツいいが 私達が新しく自分の子供用に買ったやつを使って欲しいと旦那さんから伝えたら ものすごい勢いでキレられました   お義母さんはずっと定年退職までバリバリ保育士としてやってこられてて やり方はかなり今風ではなくかなり昔のやり方で   かなり子供に関してはプライドが高く 自分の経験しか筋を通さない感じなので    キレてる時に最終的に下着とか新しいの持ってきたらいいけれど 今貯めれるとこを貯めないともったいないのに下着とかどんどん新しいのを買って お前達の給料でこれから2人の子供を育てていけるのか そんな甲斐性が私達にあるのかなど   話がそんな所までいき  勿体無い精神ももちろん大事だし 子供を見ててくれることに関しては凄く感謝しています ちゃんとお礼もしています ましてや内孫だの喜んでいてはくれたので 少し離れてるから中々会えない分会わせれる時は会わしてやりたいとも思っています 会って欲しいとも思っています   だけど正直何でそこまでお義母さんに言われないといけないのか 自分の息子の事を 息子

In [ ]:
#print(model.get_topic(0))
model.visualize_barchart(top_n_topics=10)

In [ ]:
model.visualize_hierarchy()

In [ ]:
model.visualize_topics(top_n_topics=20,
                       title='<b>Intertopic Distance Map - Top 10 Topics</b>',
                       width=1200,
                       custom_labels=True)